<a href="https://colab.research.google.com/github/havaledar/ECON3740/blob/main/A3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import pandas as pd
import numpy as np


In [ ]:
df = pd.read_stata('https://raw.githubusercontent.com/havaledar/ECON3740/main/LFS-71M0001-E-2023-August_F1.dta')
df

In [13]:
# Convert 'SEX' column to categorical and get dummies
sex_dummies = pd.get_dummies(df['SEX'], prefix='gender')
sex_dummies

,gender_Male,gender_Female
0,0,1
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
103398,0,1
103399,1,0
103400,0,1
103401,0,1


In [16]:
# Take only one column as 0 and 1 are inversely related
df['gender'] = sex_dummies["gender_Female"]

In [17]:
# Display the first few rows of the DataFrame with the new 'gender' column
print(df[['SEX', 'gender']].head())

      SEX  gender
0  Female       1
1    Male       0
2    Male       0
3    Male       0
4    Male       0


In [20]:
df['lwage'] = np.log(df['HRLYEARN'])

In [21]:
df_notna = df.dropna(subset=['lwage'])

In [ ]:
# If we import the values we can use
# df_notna['sex'] = df_notna['SEX'] - 1

In [27]:
df_notna['HRLYEARN'].isna().sum()

0

In [28]:
# Group by 'sex' and calculate the average of 'HRLYEARN'
data_averages = df_notna.groupby('SEX')['HRLYEARN'].mean().reset_index(name='wage')
data_averages

,SEX,wage
0,Male,34.937034
1,Female,30.918263


In [34]:
import statsmodels.formula.api as smf

# Create a simple linear regression model
model = smf.ols('HRLYEARN ~ C(gender)', data = df_notna)
results = model.fit()

# Display the regression results
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:               HRLYEARN   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     692.6
Date:                Thu, 09 Nov 2023   Prob (F-statistic):          1.15e-151
Time:                        04:24:24   Log-Likelihood:            -2.2925e+05
No. Observations:               53443   AIC:                         4.585e+05
Df Residuals:                   53441   BIC:                         4.585e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         34.9370      0.107    326.

In [38]:
# Create the log wage regression model with categorical variables using formula notation
model = smf.ols(formula="lwage ~ C(EDUC) + C(AGE_12) + C(SEX) + C(MARSTAT) + C(PROV)", data=df_notna)
results = model.fit()

# Display the regression results
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.339
Model:                            OLS   Adj. R-squared:                  0.339
Method:                 Least Squares   F-statistic:                     855.8
Date:                Thu, 09 Nov 2023   Prob (F-statistic):               0.00
Time:                        04:29:02   Log-Likelihood:                -24221.
No. Observations:               53443   AIC:                         4.851e+04
Df Residuals:                   53410   BIC:                         4.880e+04
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
                                                      coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------

In [40]:
!pip install stargazer

In [41]:
from stargazer.stargazer import Stargazer

# Prepare stargazer table
Stargazer([results])

In [44]:
# Obtain the F-statistic, degrees of freedom, and significance level for joint hypothesis
f_statistic = results.fvalue
degrees_of_freedom_model = results.df_model
degrees_of_freedom_residual = results.df_resid
num_regressors = degrees_of_freedom_model + 1  # +1 for the constant term

# Calculate the significance level (p-value) using the F-distribution
p_value = results.f_pvalue

print(f"F-statistic: {f_statistic}")
print(f"Degrees of freedom in the model (n - k - 1): {degrees_of_freedom_model}")
print(f"Number of restrictions (k): {num_regressors}")
print(f"Significance level to reject the null hypothesis: {p_value}")

F-statistic: 855.7772114012181
Degrees of freedom in the model (n - k - 1): 32.0
Number of restrictions (k): 33.0
Significance level to reject the null hypothesis: 0.0


In [47]:
# Separate models for men and women
model_men = smf.ols('lwage ~ C(EDUC) + C(AGE_12) + C(MARSTAT) + C(PROV)', data=df_notna[df_notna['gender'] == 0])
model_women = smf.ols('lwage ~ C(EDUC) + C(AGE_12) + C(MARSTAT) + C(PROV)', data=df_notna[df_notna['gender'] == 1])

results_men = model_men.fit()
results_women = model_women.fit()

# Get SSR for each separate model
SSR_men = sum(results_men.resid ** 2)
SSR_women = sum(results_women.resid ** 2)

# Combine models for unrestricted model
model_unrestricted = smf.ols('lwage ~ C(EDUC) + C(AGE_12) + C(MARSTAT) + C(PROV) + C(gender)', data=df_notna)
results_unrestricted = model_unrestricted.fit()
SSR_unrestricted = sum(results_unrestricted.resid ** 2)

print(f"SSR for men's model: {SSR_men}")
print(f"SSR for women's model: {SSR_women}")
print(f"SSR for the unrestricted model: {SSR_unrestricted}")


SSR for men's model: 4131.543069816736
SSR for women's model: 3538.8469428914445
SSR for the unrestricted model: 7745.796037062718


In [48]:
Stargazer([results_men, results_women])

In [49]:
import statsmodels.formula.api as smf

# Estimated the restricted model on the entire sample excluding 'sex'
model_restricted = smf.ols('lwage ~ C(EDUC) + C(AGE_12) + C(MARSTAT) + C(PROV)', data=df_notna)
results_restricted = model_restricted.fit()

# Get SSR for the restricted model
SSR_restricted = sum(results_restricted.resid ** 2)

# SSR for the unrestricted model obtained in 2.g
SSR_unrestricted = SSR_men + SSR_women  # Obtained from 2.g

# Number of regressors not including 'sex'
num_regressors = len(results_unrestricted.params) - 1  # -1 for intercept

# Degrees of freedom for the unrestricted model from 2.g
degrees_freedom_unrestricted = len(df_notna) - (2 * num_regressors) - 2

# Number of restrictions to be tested
num_restrictions = num_regressors + 1  # +1 for the intercept

# Degrees of freedom for the F-test
degrees_freedom_F = degrees_freedom_unrestricted - (2 * num_restrictions)

# Calculate the F-statistic
F_statistic = ((SSR_restricted - SSR_unrestricted) / num_restrictions) / (SSR_unrestricted / degrees_freedom_F)

print(f"SSR for the restricted model: {SSR_restricted}")
print(f"Number of restrictions (q): {num_restrictions}")
print(f"Degrees of freedom in the unrestricted model: {degrees_freedom_unrestricted}")
print(f"F-statistic: {F_statistic}")


SSR for the restricted model: 8019.334024032804
Number of restrictions (q): 33
Degrees of freedom in the unrestricted model: 53377
F-statistic: 73.49219040107538
